# <p style="text-align: center;">**Oceananigans.jl**<br><br>Massively parallel non-hydrostatic ocean modeling on GPUs in Julia</p>
## <p style="text-align: center;">**GitHub repo**: [ali-ramadhan/OceanDispatch.jl](https://github.com/ali-ramadhan/OceanDispatch.jl)<br><br>**This notebook**: [ali-ramadhan/random-jupyter-notebooks](https://github.com/ali-ramadhan/random-jupyter-notebooks)</p>

---
## **Acknowledgements**
* While I did write the code, I did not do this alone!
* **EAPS**: Chris Hill, Jean-Michel Campin, John Marshall, Greg Wagner, Mukund Gupta.
* **Julia Lab**: Valentin Churavy, Peter Ahrens, Shashi Gowda, Sung Woo Jeong, Ranjan Anantharaman, Alan Edelman.
---

### **Julia Packages used**: `FFTW, CUDAnative, CUDAdrv, CuArrays, DistributedArrays, PyPlot, Makie, Interact`
---

## **Navier-Stokes equations in a non-inertial reference frame**
$$\rho \frac{D \mathbf{u}}{D t}  = -\nabla p + \mu \, \nabla^2 \mathbf{u} + \tfrac13 \mu \, \nabla (\nabla\cdot\mathbf{u}) + \rho\mathbf{g} - \rho \left( 2\mathbf{\Omega} \times \mathbf{u} + \mathbf{\Omega} \times \mathbf{\Omega} \times \mathbf{x} + \frac{d \mathbf{U}}{dt} + \frac{d\mathbf{\Omega}}{dt} \times \mathbf{x} \right)$$ where $\displaystyle \frac{D}{Dt} = \frac{\partial}{\partial t} + \mathbf{u} \cdot \nabla$ is the material derivative, $\mathbf{g}$ is the acceleration due to gravity, $\rho$ is the fluid density, $\mathbf{x}$ is the position vector, $\mathbf{u} = (u,v,w)$ is the fluid flow velocity, $\mathbf{U}$ is the frame translation velocity,  $\mu$ is the viscosity, and $\Omega$ is the frame angular velocity. Kind of like $F=ma$.
---

## **Governing equations for non-hydrostatic ocean modeling**
### To be more exact: _Rotating, incompressible, Boussinesq equations of motion_
## Horizontal momentum: $\displaystyle \frac{\partial \mathbf{u}_h}{\partial t} = \mathbf{G}_{uh} - \nabla_h p$
## Vertical momentum: $\displaystyle \frac{\partial w}{\partial t} = G_w - \frac{\partial p}{\partial z}$
## Continuity equation (mass conservation): $\displaystyle \nabla \cdot \mathbf{u} = 0$
## Temperature: $\displaystyle \frac{\partial T}{\partial t} = G_T$
## Salinity: $\displaystyle \frac{\partial S}{\partial t} = G_S$
## Equation of state: $\displaystyle \rho = \rho(T,S,p)$
where $\mathbf{u} = (\mathbf{v}_h, w) = (u,v,w)$ is the velocity, $\mathbf{v}_h = (u,v)$ is the horizontal velocity, $\nabla = (\partial_x, \partial_y, \partial_z)$ is the del operator, $\nabla_h = (\partial_x, \partial_y)$ is the horizontal del operator. The equation of state for seawater gives the density $\rho$ in terms of $T$, $S$, and $p$. $\mathbf{G}_u = (\mathbf{G}_{uh}, G_w) = (G_u, G_v, G_w)$ represents inertial, Coriolis, gravitational, forcing, and dissipation terms.
---

## **Source terms**
\begin{align}
    G_u &= -\mathbf{u} \cdot \nabla u + fv + F_u \\
    G_v &= -\mathbf{u} \cdot \nabla v - fu + F_v \\
    G_w &= -\mathbf{u} \cdot \nabla w -g \frac{\delta \rho}{\rho_0} + F_w \\
    G_T &= -\nabla \cdot (\mathbf{u} T) + F_T \\
    G_S &= -\nabla \cdot (\mathbf{u} S) + F_S
\end{align}
---

### where $f = 2\Omega\sin\phi$ is the Coriolis frequency, $\Omega$ is the rotation rate of the Earth, $\phi$ is the latitude, $g$ is the acceleration due to gravity, $\delta\rho = \rho - \rho_0$ is the deviation of the density from that of a resting ocean, and $\rho_0$ is some reference density.

## **Spatial discretization and finite volumes**
---

## **Time stepping**
---

## **Discretized operators**

### Difference operators ($\delta_x$, $\delta_y$, $\delta_z$)
### $$\delta_x(f) = f_E - f_W, \quad\quad \delta_y(f) = f_N - f_S, \quad\quad \delta_z(f) = f_T - f_B$$

In [1]:
@inline incmod1(a, n) = a == n ? one(a) : a + 1
@inline decmod1(a, n) = a == 1 ? n : a - 1

function δˣc2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[i, j, k] - f[decmod1(i,Nx), j, k]
    end
    δf
end

function δᶻc2f(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 2:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  -(f[i, j, k] - f[i, j, k-1])
    end
    @. δf[:, :, 1] = 0
    δf
end

function δˣf2c(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 1:Nz, j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  f[incmod1(i, Nx), j, k] - f[i, j, k]
    end
    δf
end

function δᶻf2c(f)
    Nx, Ny, Nz = size(f)
    δf = zeros(Nx, Ny, Nz)
    for k in 1:(Nz-1), j in 1:Ny, i in 1:Nx
        δf[i, j, k] =  -(f[i, j, k+1] - f[i, j, k])
    end
    @. δf[:, :, end] = f[:, :, end]
    δf
end

δˣc2f (generic function with 1 method)

### Averaging operators ($\overline{\;\cdotp\;}^x$, $\overline{\;\cdotp\;}^y$, $\overline{\;\cdotp\;}^z$)

### Divergence $\nabla \cdotp \mathbb{f}$

## $$\nabla \cdot \mathbf{f} = \frac{1}{V} \left[ \delta_x (A_x f_x)  + \delta_y (A_y f_y) + \delta_z (A_z f_z) \right]$$

### Flux divergences $\nabla \cdotp (\mathbf{u} Q)$

### Pressure gradient force $\nabla p$

### Laplacian diffusion $\nabla\cdotp(\kappa\nabla Q)$

### Viscous dissipation $\nabla\cdotp(\nu\nabla u)$

---

## **Elliptic problem for the pressure**

### $$\nabla^2 (\phi_s + \phi_{nh}) = \nabla \cdotp \mathbf{G}_u = \mathscr{F}$$

---

## **Fourier-spectral solver for 3D Poisson equation with horizontally periodic and vertically Neumann boundary conditions**
### $$ \hat{\phi}_{nh} = -\frac{\hat{\mathscr{F}}}{\mathbf{k}^2} $$
where $\mathbf{k}^2 = k_x^2 + k_y^2 + k_z^2$
---

## **Profiling results from a similar MITgcm simulation**

## **References**
### 1. Marshall, J., Hill, C., Perelman, L., Adcroft, A., **Hydrostatic, quasi-hydrostatic, and nonhydrostatic ocean modeling**, _Journal of Geophysical Research_ 102(C3), 5733-5752 (1997).
### 2. Marshall, J., Adcroft, A., Hill, C., Perelman, L., & Heisey, C., **A finite-volume, incompressible Navier Stokes model for studies of the ocean on parallel computers**, _Journal of Geophysical Research_ 102(C3), 5753-5766 (1997).
### 3. Alistair Adcroft, Jean-Michel Campin, Ed Doddridge, Stephanie Dutkiewicz, Constantinos Evangelinos, David Ferreira, Mick Follows, Gael Forget, Baylor Fox-Kemper, Patrick Heimbach, Chris Hill, Ed Hill, Helen Hill, Oliver Jahn, Jody Klymak, Martin Losch, John Marshall, Guillaume Maze, Matt Mazloff, Dimitris Menemenlis, Andrea Molod, and Jeff Scott. **MITgcm user manual**. (2008-2018)
### 4. Frigo, M., & Johnson, S. G., **The design and implementation of FFTW3**, _Proceedings of the IEEE_, 93(2), 216-231 (2005).

In [1]:
cd("C:\\Users\\Ali\\Documents\\Git\\OceanDispatch.jl\\")

In [2]:
using Pkg
using Statistics: mean

Pkg.activate(".");

In [3]:
using OceanDispatch

In [ ]:
Pkg.test("OceanDispatch")

In [4]:
Nx, Ny, Nz = 10, 10, 10
f = rand(Nx, Ny, Nz)
f .= f .- mean(f)
ϕ = solve_poisson_3d_ppn(f, Nx, Ny, Nz)
laplacian3d_ppn(ϕ) ≈ f

false

In [ ]:
f = rand(Nx, Ny, Nz)
f .= f .- mean(f)
ϕ = solve_poisson_3d_ppn(f, Nx, Ny, Nz)
laplacian3d_ppn(ϕ) ≈ f